In [1]:
!pip install -U sentence-transformers --use-feature=2020-resolver
!pip install razdel
!pip install navec
!pip install slovnet

     |████████████████████████████████| 64 kB 511 kB/s eta 0:00:011
     |████████████████████████████████| 1.3 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 5.4 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-py3-none-any.whl size=101031 sha256=a862189fc753cbf87937ddcce56f2a4d27e271d98752d3dbe778266409479586
  Stored in directory: /root/.cache/pip/wheels/b0/57/6e/72164eb7d28256df352bcce26174d9133d191c232006fb13d2
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2
ERROR: allennlp 1.1.0 requires transformers<3.1,>=3.0, but you'll have transformers 3.5.1 which is incompatible.
Y

In [2]:
import json
import numpy as np
import pandas as pd
import os
import re
import torch
from os import listdir
from os.path import isfile, join
from navec import Navec
from razdel import sentenize, tokenize
from sentence_transformers import SentenceTransformer, util, models
from slovnet import Morph

In [3]:
threshold = 0.90
queries_path = '../input/specifications-jsons/' # путь к запросам
handbook_path = '../input/handbook/handbook.xls' # путь к справочнику 
model_path_directory = '../input/pytorch-rubert/' # путь к модели
results_path = './' # путь к csv

In [4]:
if not os.path.isfile('navec_news_v1_1B_250K_300d_100q.tar'):
    !wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
if not os.path.isfile('slovnet_morph_news_v1.tar'):
    !conda install -y gdown 
    !gdown https://drive.google.com/uc?id=1U4TT9cGtdYL8I0G5NSd2gCvia9wpFpDr

--2020-12-03 07:19:49--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  38.5MB/s    in 0.7s    

2020-12-03 07:19:50 (38.5 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8         

In [5]:
jsons = [file for file in listdir(queries_path) if isfile(join(queries_path, file))][:20]

In [6]:
handbook = pd.read_excel(handbook_path)
handbook.head()

,Код,Артикул,Наименование,Полное наименование,Вид номенклатуры,Базовая единица измерения,Единица хранения остатков,Номер ГТД,Страна происхождения,Ставка НДС,Номенклатурная группа,Ценовая группа,Комментарий
0,1920,NaN,"500х23,9 труба ПНД ""техническая""","Труба ПНД ""техническая"" 500х23,9",Товар,м,м,NaN,NaN,20%,NaN,NaN,NaN
1,УТ000004069,NaN,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
2,УТ000000195,NaN,003L0280 Клапан RLV-K G 3/4A; G 1/2А,"003L0280 Н-образный клапан RLV-K, прямой, для ...",Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
3,УТ000000198,NaN,003L7603 Регулятор перепада давления ASV-PV Ду25,003L7603 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
4,УТ000001434,NaN,003L7604 Регулятор перепада давления ASV-PV Ду40,003L7604 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN


In [7]:
# замена пустых значений Полного Наименования на значений Наименовани
handbook.loc[handbook['Полное наименование'].isnull(), 'Полное наименование'] = handbook['Наименование']

In [8]:
# Corpus with example sentences
handbook_names = handbook.iloc[:, 3].tolist()

In [9]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)

Morph(
    infer=MorphInfer(
        model=Morph(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                       

## RuBERT

In [10]:
word_embedding_model = models.Transformer(model_path_directory, max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

rubert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [11]:
handbook_rubert_embeddings = rubert_model.encode(handbook_names, convert_to_tensor=True)

In [12]:
def element_extraction(text):
    digit_terms = re.findall('\w*\d*[\.,]?\d+\w*', text)
    chunk = []
    for sentence in sentenize(text.lower()):
        print(sentence)
        tokens = [_.text for _ in tokenize(sentence.text)]
        chunk.append(tokens)
    markup = next(morph.map(chunk))
    nouns = []
    adjectives = []
    for token in markup.tokens:
        if (token.pos == 'NOUN' or token.pos == 'PNOUN') and not(re.findall('\d+', token.text)):
            nouns.append(token.text)
        if token.pos == 'ADJ' or token.pos == 'ADV':
            adjectives.append(token.text)
    return digit_terms, nouns, adjectives

In [13]:
def improved_cosine_similarity(record, addendums_coef, adj_coef):
    digit_terms, nouns, adjectives = element_extraction(record)
    digit_terms_encoded = rubert_model.encode(' '.join(digit_terms), convert_to_tensor=True)
    nouns_encoded = rubert_model.encode(' '.join(nouns), convert_to_tensor=True)
    adjectives_encoded = rubert_model.encode(' '.join(adjectives), convert_to_tensor=True)
    
    digits_cos_scores = util.pytorch_cos_sim(query_digit_terms_encoded, digit_terms_encoded).cpu()
    nouns_cos_scores = util.pytorch_cos_sim(query_nouns_encoded, nouns_encoded).cpu()
    adj_cos_scores = util.pytorch_cos_sim(query_adjectives_encoded, adjectives_encoded).cpu()
    
    return addendums_coef * (digits_cos_scores + nouns_cos_scores + adj_coef * adj_cos_scores)

In [14]:
hashed_requests = {}

In [22]:
%%time
top_k = 20
addendums_coef = 0.1
adj_coef = 0.8

for json_filename in jsons:
    
    results = {'Исходный текст': [],
               'Количество': [],
               'Единицы измерения': [],
               'Предлагаемый вариант': [],
               'Коэффициент сходства, %': []}

    with open(queries_path + json_filename) as json_file:
        data = json.load(json_file)
    
    queries = [(item['Item-Model'].strip(),  item['Quantity'].strip(), item['Measure'].strip())
               for group in data
               for item in group['Items']
               if item['Item-Model']]
    
    for query in queries:
        if query[0] in hashed_requests:
            if float(hashed_requests[query[0]][1]) >= threshold:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append(hashed_requests[query[0]][0])
                results['Коэффициент сходства, %'].append(hashed_requests[query[0]][1])
            else:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append('')
                results['Коэффициент сходства, %'].append('')
        else:
        
            query_embedding = rubert_model.encode(query[0], convert_to_tensor=True)
            cos_scores = util.pytorch_cos_sim(query_embedding, handbook_rubert_embeddings)[0]
            cos_scores = cos_scores.cpu()

            top_results = torch.topk(cos_scores, k=top_k)
            top_results_values = top_results.values.numpy()
            top_results_indices = top_results.indices.numpy()

            query_digit_terms, query_nouns, query_adjectives = element_extraction(query[0])
            query_digit_terms_encoded = rubert_model.encode(' '.join(query_digit_terms), convert_to_tensor=True)
            query_nouns_encoded = rubert_model.encode(' '.join(query_nouns), convert_to_tensor=True)
            query_adjectives_encoded = rubert_model.encode(' '.join(query_adjectives), convert_to_tensor=True)
            top_results_values = [top_results_values[n] + improved_cosine_similarity(handbook_names[top_results_indices[n]],
                                                                                     addendums_coef,
                                                                                     adj_coef) for n in range(top_k)]
            max_cos_indices = np.argsort(top_results_values)[::-1]
            
            new_top_results_values = [top_results_values[n] for n in max_cos_indices][:1:]
            top_results_indices = [top_results_indices[n] for n in max_cos_indices][:1:]
            self_similarity = 1 + addendums_coef * (2 + adj_coef)

            similarity_coef = float(new_top_results_values[0][0][0]) / self_similarity
            
            suggested_option = handbook_names[top_results_indices[0]]
            
            if similarity_coef >= threshold:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append(suggested_option)
                results['Коэффициент сходства, %'].append(similarity_coef * 100)
                hashed_requests.update({query[0]: (suggested_option, similarity_coef * 100)})
            else:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append('')
                results['Коэффициент сходства, %'].append('')
                hashed_requests.update({query[0]: (suggested_option, similarity_coef * 100)})

    results = pd.DataFrame(results)
    results.to_csv(results_path + json_filename.strip('.json') + '.csv', sep=';', encoding='utf-8-sig')


Substring(0, 16, 'грунтовка гф-021')



Substring(0, 31, 'седелка компрессионная 025х1/2"')



Substring(0, 30, 'отвод компрессионный нр 025х1"')



Substring(0, 29, 'муфта компрессионная нр 25х1"')



Substring(0, 30, 'отвод компрессионный вр 025х1"')


...


Substring(0, 51, 'шток фиксированной длины 9000а 1.5 м. dn 150, hawle')



Substring(0, 22, 'шпатель фасадный 350мм')



Substring(0, 61, 'втулка под фланец с патрубком пэ100 - 110 sdr17, удл. l=600 м')



Substring(0, 53, 'шток фиксированной длины 9000е2 1.25 м. dn 350, hawle')



CPU times: user 6min 13s, sys: 12.5 s, total: 6min 25s
Wall time: 3min 9s


In [23]:
!pip install faiss

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [24]:
handbook_rubert_embeddings = np.loadtxt('../input/handbook-embeddings/array.txt').astype('float32')

In [25]:
import faiss
n_dimensions = handbook_rubert_embeddings.shape[1]
quantiser = faiss.IndexFlatIP(n_dimensions) 
index = faiss.IndexIVFFlat(quantiser, n_dimensions, 1910)
index.nprobe = 1910
index.train(handbook_rubert_embeddings)
index.add(handbook_rubert_embeddings)

In [26]:
def improved_cosine_similarity(record, addendums_coef, adj_coef):
    digit_terms, nouns, adjectives = element_extraction(record)
    digit_terms_encoded = rubert_model.encode(' '.join(digit_terms), convert_to_tensor=False)
    nouns_encoded = rubert_model.encode(' '.join(nouns), convert_to_tensor=False)
    adjectives_encoded = rubert_model.encode(' '.join(adjectives), convert_to_tensor=False)
    
    digits_cos_scores = util.pytorch_cos_sim(query_digit_terms_encoded, digit_terms_encoded).cpu()
    nouns_cos_scores = util.pytorch_cos_sim(query_nouns_encoded, nouns_encoded).cpu()
    adj_cos_scores = util.pytorch_cos_sim(query_adjectives_encoded, adjectives_encoded).cpu()
    
    return addendums_coef * (digits_cos_scores + nouns_cos_scores + adj_coef * adj_cos_scores)

In [27]:
hashed_requests = {}

In [28]:
%%time

top_k = 10
addendums_coef = 0.1
adj_coef = 0.8

self_similarity = 1 + addendums_coef * (2 + adj_coef)

for json_filename in jsons:
    
    results = {'Исходный текст': [],
               'Количество': [],
               'Единицы измерения': [],
               'Предлагаемый вариант': [],
               'Коэффициент сходства': []}

    with open(queries_path + json_filename) as json_file:
        data = json.load(json_file)
    
    queries = [(item['Item-Model'].strip(),  item['Quantity'].strip(), item['Measure'].strip())
               for group in data
               for item in group['Items']
               if item['Item-Model']]

    for query in queries:
        if query[0] in hashed_requests:
            if float(hashed_requests[query[0]][1]) >= threshold:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append(hashed_requests[query[0]][0])
                results['Коэффициент сходства'].append(hashed_requests[query[0]][1])
            else:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append('')
                results['Коэффициент сходства'].append('')
        else:
        
            query_embedding = rubert_model.encode(query[0], convert_to_tensor=False)

            _, top_k_indices = index.search(query_embedding.reshape(1, 768), top_k)
            top_k_indices = top_k_indices[0]

            query_digit_terms, query_nouns, query_adjectives = element_extraction(query[0])
            query_digit_terms_encoded = rubert_model.encode(' '.join(query_digit_terms), convert_to_tensor=False)
            query_nouns_encoded = rubert_model.encode(' '.join(query_nouns), convert_to_tensor=False)
            query_adjectives_encoded = rubert_model.encode(' '.join(query_adjectives), convert_to_tensor=False)
            
            top_results_values = np.squeeze(handbook_rubert_embeddings[top_k_indices])
            cos_scores = util.pytorch_cos_sim(top_results_values, query_embedding).cpu().numpy().ravel()
            top_results_values = [cos_scores[n] + improved_cosine_similarity(handbook_names[top_k_indices[n]],
                                                                             addendums_coef,
                                                                             adj_coef) for n in range(top_k)]
            top_names = [handbook_names[n_index] for n_index in top_k_indices]
            max_value = max(top_results_values)
            suggested_option = top_names[top_results_values.index(max_value)]

            similarity_coef = float(max_value[0][0]) / self_similarity
            if similarity_coef >= threshold:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append(suggested_option)
                results['Коэффициент сходства'].append(similarity_coef)
                hashed_requests.update({query[0]: (suggested_option, similarity_coef)})
            else:
                results['Исходный текст'].append(query[0])
                results['Количество'].append(query[1])
                results['Единицы измерения'].append(query[2])
                results['Предлагаемый вариант'].append('')
                results['Коэффициент сходства'].append('')
                hashed_requests.update({query[0]: (suggested_option, similarity_coef)})

    results = pd.DataFrame(results)
    results.to_csv(results_path + json_filename.strip('.json') + '.csv', sep=';', encoding='utf-8-sig')


Substring(0, 33, 'труба 114х4.0 12x18h10t нерж. ст.')



Substring(0, 24, 'труба пвх 110х3,0х2000мм')



Substring(0, 47, 'труба пвх раструбная класса "н" 110х3,2х560 sn8')



Substring(0, 28, 'труба пвх 110х3,2х2000мм sn4')



Substring(0, 25, 'труба пвх 110х3,2х560 sn4')


...


Substring(0, 22, 'шпатель фасадный 350мм')



Substring(0, 61, 'втулка под фланец с патрубком пэ100 - 160 sdr17, удл. l=300 м')



Substring(0, 62, 'втулка под фланец с патрубком пэ100 - 355 sdr17, удл. l=1000 м')



Substring(0, 69, 'втулка под фланец сварная с патрубком пэ100 - 110 sdr17, удл. l=400 м')



CPU times: user 13min 7s, sys: 27.1 s, total: 13min 35s
Wall time: 6min 40s
